<a href="https://colab.research.google.com/github/psokhi/Centralized-repository-shipping_calculations/blob/main/langchain_tutorial2_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [ ]:
# Using secrets
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

**Basic**

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

In [ ]:
import datetime

# Get current date and time
now = datetime.datetime.now()
print("Current date and time:", now)

# Get current time only
current_time = now.strftime("%H:%M:%S")
print("Current time:", current_time)

# Get current date only
current_date = now.strftime("%Y-%m-%d")
print("Current date:", current_date)

Current date and time: 2025-05-06 00:43:07.213582
Current time: 00:43:07
Current date: 2025-05-06


In [ ]:
def get_current_time(*args,**kwargs):
  import datetime
  now = datetime.datetime.now()
  current_time = now.strftime("%H:%M:%S")
  return current_time

In [ ]:
# get prompt template from hub
# React - Reason and Action
# https://smith.langchain.com/hub/hwchase17/react

"""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

#prompt = hub.pull("hwchase17/react")
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("hwchase17/react", include_model=False)

In [ ]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [ ]:
tools = [Tool(name="Time",
              func=get_current_time,
              description="Useful for when you need to know the current time")
]
llm = ChatOpenAI()
agent = create_react_agent(llm=llm,tools=tools,prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
    )

result = agent_executor.invoke({"input":"What time is it now?"})
print(result)



> Entering new AgentExecutor chain...
I should use the Time tool to check the current time.
Action: Time
Action Input: None01:22:28I have the current time now.
Final Answer: 01:22:28

> Finished chain.
{'input': 'What time is it now?', 'output': '01:22:28'}


**React Chat**

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=ef33e8da5eaa1d467e0b16c647124679cb834c948d6cd44c58d0c9fac844322f
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
def search_wikipedia(query):
  from wikipedia import summary
  try:
    return summary(query,sentences=2)
  except:
    return "I couldn't find any information on that."

def get_current_date(*args,**kwargs):
  import datetime
  now = datetime.datetime.now()
  current_date = now.strftime("%Y-%m-%d")
  return current_date

In [ ]:
get_current_date()

'2025-05-07'

In [ ]:
from wikipedia import summary
query = "Sachin Tendulkar"
summary(query,sentences=2)

'Sachin Ramesh Tendulkar ( ; Marathi: [sətɕin t̪eɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest cricketers of all time, and is the holder of several world records, including being the all-time highest run-scorer in both ODI and Test cricket, receiving the most player of the match awards in international cricket, and being the only batsman to score 100 international centuries.'

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("hwchase17/structured-chat-agent", include_model=False)

In [ ]:
tools = [
    Tool(name="Date",
         func=get_current_date,
         description="Useful for when you need to know the current date"),
    Tool(name="Wikipedia",
         func=search_wikipedia,
         description="Useful for when you need to know information about a topic"),
    ]

memory = ConversationBufferMemory(memory_key="chat_history",
                                  return_messages=True)

# set initial message in chat memory for context
initial_message = "You are a AI assistant that can provide helpful \
answers using available tools. If you are unable to answer then \
say 'I don't know about this.'"

memory.chat_memory.add_message(SystemMessage(content=initial_message))

llm = ChatOpenAI()
agent = create_structured_chat_agent(llm=llm,tools=tools,prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory, # use the conv history to maintain context
    handle_parsing_errors=True, # handle any errors gracefully
    max_iter=10
    )

In [ ]:
while True:
  print ("Start chatting. Print 'Exit' to stop.")
  user_input = input("User: ")
  if user_input.lower() == "exit":
    break
  memory.chat_memory.add_message(HumanMessage(content=user_input))
  response = agent_executor.invoke({"input":user_input})
  print("Bot: ",response["output"])
  memory.chat_memory.add_message(AIMessage(content=response["output"]))

Start chatting. Print 'Exit' to stop.
User: When was Sachin Tendulkar born and how old is he?


> Entering new AgentExecutor chain...
Could not parse LLM output: ```
{
  "action": Wikipedia,
  "action_input": "Sachin Tendulkar"
}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse LLM output: I don't know about this.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse LLM output: I don't know about this.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse LLM output: I don't know about this.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse LLM output: I don't know about this.

R

**Agents and RAG**

In [ ]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.1 MB/s eta 0:

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os

In [ ]:
current_dir = os.getcwd()
file_path = os.path.join(current_dir,"the-verdict.txt")
persistent_dir = os.path.join(current_dir,"chroma_agent")

In [ ]:
# check if vector store already exists
if not os.path.exists(persistent_dir):
  print("---Initializing vector store---")

  loader = TextLoader(file_path=file_path)
  documents = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=400,
      chunk_overlap=20
  )
  docs = text_splitter.split_documents(documents)
  db = Chroma.from_documents(docs,
                             OpenAIEmbeddings(),
                             persist_directory=persistent_dir)
  print("---Vector store created---")
else:
  print("---Vector store exists---")

---Initializing vector store---
---Vector store created---


In [ ]:
db = Chroma(persist_directory=persistent_dir,
            embedding_function=OpenAIEmbeddings())
llm = ChatOpenAI()
retriever=db.as_retriever()

In [ ]:
# Contextualize system prompt
# This system prompt helps the AI understand
# that it should reformulate the question
# based on the chat history to make it
# into a standalone question.
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat hsitory, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [("system", contextualize_q_system_prompt),
     MessagesPlaceholder("chat_history"),
     ("human","{input}")]
)

qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use 3 sentences maximum and keep the answer short."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [("system",qa_system_prompt),
     MessagesPlaceholder("chat_history"),
     ("human","{input}")]
)

In [ ]:
# create a history-aware retriever
history_aware_retreiver = create_history_aware_retriever(
    llm,retriever,contextualize_q_prompt
)

# create a chain to combine documents for question answering
question_answer_chain = create_stuff_documents_chain(
    llm,qa_prompt)

# create a retrieval chain
rag_chain = create_retrieval_chain(
    history_aware_retreiver,question_answer_chain)

In [ ]:
from langsmith import Client
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
react_docstore_prompt = client.pull_prompt("hwchase17/react")

In [ ]:
tools = [
    Tool(
        name="Answer Question",
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input":input,"chat_history":kwargs.get("chat_history",[])}
            ),
        description="useful for when you need to answer questions about the context"
        )
]

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True
)

chat_history = []
while True:
  query = input("\n Ask a question (or 'Exit): ")
  if query.lower() == "exit":
    break
  response = agent_executor.invoke({"input":query, "chat_history":chat_history})
  print(f"AI: {response['output']}")

  chat_history.append(HumanMessage(content=query))
  chat_history.append(AIMessage(content=response["output"]))


 Ask a question (or 'Exit): why did Jack Gisburn stop painting? Limit answer to not more than 3 sentences.
AI: Jack Gisburn stopped painting in the height of his glory, not due to his wife, but for reasons that remained a tempting problem for others to ponder on. His decision to give up painting was considered final and irrevocable by those around him, and the price of his paintings increased after he ceased painting.

 Ask a question (or 'Exit): Who finally painted the portrait of the deceased artist?
AI: I don't know.

 Ask a question (or 'Exit): What was the name of the deceased artist?
AI: The deceased artist's name was Jack.

 Ask a question (or 'Exit): exit


**Build your own Tools**

Tool constructor

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI

In [ ]:
def greet_user(name: str) -> str:
  return f"Hello, {name}!"

def reverse_string(text: str) -> str:
  return text[::-1]

def concat_strings(a: str, b: str) -> str:
  return a + b

# Pydantic model for tool arguments
class ConcatenateStringsArgs(BaseModel):
  a: str = Field(description="First string")
  b: str = Field(description="Second string")

In [ ]:
tools = [
    # Use Tool() for simple functions with single arg
    Tool(
        name="GreetUser",
        func=greet_user,
        description="Greets the user by name"),
    Tool(
        name="ReverseString",
        func=reverse_string,
        description="Reverses a string"),
    # Use StructuredTool that use funcctions with multiple args
    StructuredTool.from_function(
        name="ConcatenatesStrings",
        func=concat_strings,
        description="Concatenates two strings",
        args_schema=ConcatenateStringsArgs
    )
]

In [ ]:
from langsmith import Client
import os
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("hwchase17/openai-tools-agent")

In [ ]:
llm = ChatOpenAI()
agent = create_tool_calling_agent(llm,tools,prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent,tools,verbose=True)
response = agent_executor.invoke({"input":"Greet Alice"})
print(f"Response for greetings:{response['output']}")

response = agent_executor.invoke({"input":"Reverse the string 'hello'"})
print(f"Response for string reversal:{response['output']}")

response = agent_executor.invoke({"input":"Concatenate 'hello' and ' world' "})
print(f"Response for string concat:{response['output']}")



> Entering new AgentExecutor chain...

Invoking: `GreetUser` with `Alice`


Hello, Alice!Alice has been greeted.

> Finished chain.
Response for greetings:Alice has been greeted.


> Entering new AgentExecutor chain...

Invoking: `ReverseString` with `hello`


ollehThe reversed string of 'hello' is 'olleh'.

> Finished chain.
Response for string reversal:The reversed string of 'hello' is 'olleh'.


> Entering new AgentExecutor chain...

Invoking: `ConcatenatesStrings` with `{'a': 'hello', 'b': ' world'}`


hello worldThe result of concatenating 'hello' and ' world' is 'hello world'.

> Finished chain.
Response for string concat:The result of concatenating 'hello' and ' world' is 'hello world'.


Notice the last execution how the two strings were correctly identified.

Base Tool

In [ ]:
from typing import Type
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

In [ ]:
# Pydantic model for tool arguments
class SimpleSearchInput(BaseModel):
  query: str = Field(description="should be a search query")

class MultiplyNumbersArgs(BaseModel):
  x: float = Field(description="First number to multiply")
  y: float = Field(description="Second number to multiply")

In [ ]:
# Custom tool with 1 custom input
class SimpleSearchTool(BaseTool):
  name: str = "simple_search"
  description: str = "useful for when need to answer about a topic"
  args_schema: Type[BaseModel] = SimpleSearchInput

  def _run(self,query: str) -> str:
    from wikipedia import summary
    try:
      result = summary(query,sentences=2)
      return f"Search results for: {query}\n\n\n{result}\n"
    except:
      return f"Search results for: {query}\n\n\n Sorry, I don't know.\n"

In [ ]:
# Custom tool with 2 custom input
class MultiplyNumbersTool(BaseTool):
  name: str = "multiply_numbers"
  description: str = "useful for multiplying two numbers"
  args_schema: Type[BaseModel] = MultiplyNumbersArgs

  def _run(self, x: float, y: float) -> str:
    result = x * y
    return f"The product of {x} and {y} is {result}"

In [ ]:
from langsmith import Client
import os
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("hwchase17/openai-tools-agent")

In [ ]:
# create tools using Pydantic subclass approach
tools = [
    SimpleSearchTool(),
    MultiplyNumbersTool()
]

llm = ChatOpenAI()
agent = create_tool_calling_agent(llm,tools,prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent,tools,verbose=True)
response = agent_executor.invoke({"input":"Who wrote 'Alice in Wonderland'?"})
print(f"Response for topic search:{response['output']}")

response = agent_executor.invoke({"input":"Multiply 3.14 and 1.414"})
print(f"Response for multiplication:{response['output']}")



> Entering new AgentExecutor chain...

Invoking: `simple_search` with `{'query': 'Author of Alice in Wonderland'}`


Search results for: Author of Alice in Wonderland


Alice in Wonderland Syndrome (AIWS), also known as Todd's Syndrome or Dysmetropsia, is a neurological disorder that distorts perception. People with this syndrome may experience distortions in their visual perception of objects, such as appearing smaller (micropsia) or larger (macropsia), or appearing to be closer (pelopsia) or farther (teleopsia) than they are.
It seems there was some confusion in the search results. The author of "Alice in Wonderland" is Lewis Carroll. If you have any more questions or need further assistance, feel free to ask!

> Finished chain.
Response for topic search:It seems there was some confusion in the search results. The author of "Alice in Wonderland" is Lewis Carroll. If you have any more questions or need further assistance, feel free to ask!


> Entering new AgentExecutor chain...

In